In [1]:
import os

In [2]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.3 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/jf3511/CCM-final-project.git

Cloning into 'CCM-final-project'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 311 (delta 113), reused 267 (delta 69), pack-reused 0
Receiving objects: 100% (311/311), 7.94 MiB | 19.69 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [4]:
cd CCM-final-project

/content/CCM-final-project


In [5]:
!git pull origin main

From https://github.com/jf3511/CCM-final-project
 * branch            main       -> FETCH_HEAD
Already up to date.


In [6]:
print(f"Current directory {os.getcwd()}")

Current directory /content/CCM-final-project


In [7]:
!pip install -e .

Obtaining file:///content/CCM-final-project
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ple


In [8]:
!pip install pygame

In [9]:
!pip install -q tensorflow

In [10]:
!pip install -q keras

In [11]:
pip install screeninfo

In [12]:
import os
import pickle
import random
import platform
import sys
import time, datetime
from collections import deque
from pprint import pprint

import keras
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
from keras import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from ple import PLE
from ple.games.speed_boost_human import Pixelcopter
#from win32api import GetSystemMetrics

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version} {platform.system()}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU')) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

<ipython-input-12-3d00627678ab>:19: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom
Tensor Flow Version: 2.15.0
Keras Version: 2.15.0

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] Linux
Pandas 2.0.3
Scikit-Learn 1.2.2
GPU is NOT AVAILABLE


In [13]:
x = 1200  # Example x-coordinate
y = 200   # Example y-coordinate

# Set the SDL video window position environment variable
os.environ['SDL_VIDEO_WINDOW_POS'] = f"{x},{y}"

In [14]:
# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self.log_write_dir = self.log_dir
        self._train_dir = os.path.join(self.log_dir + 'train')
        self._val_dir = os.path.join(self.log_dir, 'validation')
        self._should_write_train_graph = False

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        self.model = model
        self._train_step = self.model._train_counter
        self._val_step = self.model._test_counter

    # Overrides, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrides
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrides, so won't close writer
    def on_train_end(self, logs=None):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        with self.writer.as_default():
            for key, value in stats.items():
                tf.summary.scalar(key, value, step=self.step)
                self.writer.flush()

In [15]:
# DQN agent class
class DQNAgent:
    # hyper-parameters
    LEARNING_RATE = 1e-3
    BATCH_SIZE = 64
    INPUT_SIZE = 7
    OUTPUT_SIZE = 2
    EPSILON = 1
    DECAY_RATE = 0.005
    MIN_EPSILON = 0.1
    GAMMA = 0.99
    MEMORY_SIZE = 1000
    UPDATE_TARGET_LIMIT = 5
    MODEL_NAME = f"DQN model LR={LEARNING_RATE} BATCH={BATCH_SIZE} MEM_SIZE={MEMORY_SIZE}"
    #LOAD_MODEL = keras.models.load_model("/content/sample_data/DQN model LR=0.001 BATCH=64 MEM_SIZE=1000____-3.00max___-3.41avg___-4.00min.h5")

    # based on documentation, state has 7 features
    # output is 2 dimensions, 0 = do nothing, 1 = jump

    def __init__(self, mode="train"):
        # depending on what mode the agent is in, will determine how the agent chooses actions
        # if agent is training, EPSILON = 1 and will decay over time with epsilon probability of exploring
        # if agent is playing (using trained model), EPSILON = 0 and only choose actions based on Q network
        self.EPSILON = 1 if mode == "train" else 0
        print(self.EPSILON)
        # main model  # gets trained every step
        self.model = self.create_model()
        print(self.model.summary())
        print("Finished building baseline model..")
        self.action_map = {
            0: None,
            1: 119
        }
        # Target model this is what we .predict against every step
        self.target_model = self.create_model()
        print("Finished building target model..")
        self.target_model.set_weights(self.model.get_weights())

        self.replay_memory = deque(maxlen=self.MEMORY_SIZE)
        self.tensorboard = ModifiedTensorBoard(log_dir=f"log_speed_boost/{self.MODEL_NAME}-{int(time.time())}")
        self.target_update_counter = 0
        self.rewards = []

    def create_model(self):
        model = Sequential()

        model.add(Dense(32, input_shape=(self.INPUT_SIZE,), activation="relu"))

        model.add(Dense(64, activation="relu"))
        model.add(Dropout(0.2))

        model.add(Dense(64, activation="relu"))
        model.add(Dropout(0.2))

        model.add(Dense(self.OUTPUT_SIZE, activation='linear'))  # ACTION_SPACE_SIZE = how many choices (9)
        model.compile(loss="mse", optimizer=Adam(lr=self.LEARNING_RATE), metrics=['accuracy'])
        return model

    def update_replay_memory(self, state, action, reward, next_state, done):
    # Store the experience in memory
        self.replay_memory.append((state, action, reward, next_state, done))

    def select_action(self, state):
    # Choose random action with probability epsilon
        if np.random.uniform() < self.EPSILON:
            action_index = np.random.randint(self.OUTPUT_SIZE)
    # Otherwise, choose epsilon-greedy action from neural net
        else:
            action_index = self.get_predicted_action([state])
        actual_action = self.action_map[action_index]
        return action_index, actual_action

    def get_qs(self, state, step):
        return self.model.predict(np.array(state))[0]

    def construct_memories(self):
        # Get a minibatch of random samples from memory replay table
        replay = random.sample(self.replay_memory, self.BATCH_SIZE)
        # Get current states from minibatch, then query NN model for Q values
        states = np.array([step[0] for step in replay])
        Q = self.model.predict(states)
        # Get future states from minibatch, then query NN model for Q values
        new_states = np.array([step[3] for step in replay])
        Q_next = self.model.predict(new_states)

        X = []
        Y = []

        for index, (state, action, reward, state_, done) in enumerate(replay):
            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_Q = np.amax(Q_next[index])
                new_Q = reward + self.GAMMA * max_Q
            else:
                new_Q = reward

            # Update the Q value for given state
            target = Q[index]
            target[action] = new_Q

            # Append new values to training data
            X.append(state)
            Y.append(target)
        return X, Y

    def train(self, is_terminal, step):
        if not os.path.isdir('models_speed_boost'):
            os.makedirs('models_speed_boost')

        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < self.MEMORY_SIZE:
            return

        # constructs training data for training of the neural network
        X, y = self.construct_memories()

        self.model.fit(np.array(X), np.array(y), batch_size=self.BATCH_SIZE, verbose=1, shuffle=False,
                       callbacks=[self.tensorboard] if is_terminal else None)

        # Update target network counter after every episode
        if is_terminal:
            # self.model.fit(np.array(X), np.array(y), batch_size=self.BATCH_SIZE, verbose=1, shuffle=False, callbacks=[self.tensorboard])
            self.target_update_counter += 1

        # If counter reaches a set value, update the target network with weights of main network
        if self.target_update_counter > self.UPDATE_TARGET_LIMIT:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    def get_predicted_action(self, sequence):
        prediction = self.model.predict(np.array(sequence))[0]
        print("Prediction", prediction)
        return np.argmax(prediction)

In [16]:
def start():
    game = Pixelcopter(width=250, height=250)
    env = PLE(game, fps=30, display_screen=True, force_fps=True)
    env.init()
    episode_rewards = []
    agent = DQNAgent("train")
    num_episodes = 5000
    interval = 100
    print("State attributes", env.getGameState().keys())
    print("All actions", env.getActionSet())
    for episode in range(1, num_episodes + 1):
        agent.tensorboard.step = episode
        done = False
        step = 1
        total_reward = 0.0
        # initial state
        state = np.array(list(env.getGameState().values()))
        print("State:", state)
        while not done:
            if env.game_over():
                print("GAME OVER!")
                done = True
            action_index, action = agent.select_action(state)
            action_string = 'jump!' if action_index == 1 else 'chill'
            print("Action:", action, action_string)
            reward = env.act(action)
            print("Reward:", reward)
            new_state = np.array(list(env.getGameState().values()))
            # update total reward
            total_reward += reward
            # update replay memory
            agent.update_replay_memory(state, action_index, reward, new_state, done)
            # update q_network
            agent.train(done, step)
            # update current state with new state
            state = new_state
            # increment time step
            step += 1
        # Append episode rewards to list of all episode rewards
        episode_rewards.append(total_reward)
        can_update = episode % interval
        print(can_update)
        if not can_update or episode == 1:
            average_reward = np.mean(episode_rewards[-interval:])
            min_reward = np.min(episode_rewards[-interval:])
            max_reward = np.max(episode_rewards[-interval:])
            agent.tensorboard.update_stats(
                reward_avg=average_reward,
                reward_min=min_reward,
                reward_max=max_reward,
                epsilon=agent.EPSILON
            )
            # Save model, but only when min reward is greater or equal a set value
            model_folder = datetime.datetime.now().strftime("%d-%m-%Y %H%M%S")
            agent.model.save(
                f'models_speed_boost/{model_folder}/{agent.MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min.h5')
        # Decay epsilon
        if agent.EPSILON > agent.MIN_EPSILON:
            agent.EPSILON *= agent.DECAY_RATE
            # ensure epsilon does not subside below minimum value
            agent.EPSILON = max(agent.MIN_EPSILON, agent.EPSILON)
        env.reset_game()

In [17]:
start()

1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                256       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 6658 (26.01 KB)
Trainable params: 6658 (26

None
Finished building baseline model..
Finished building target model..
State attributes dict_keys(['player_y', 'player_vel', 'player_dist_to_ceil', 'player_dist_to_floor', 'next_gate_dist_to_player', 'next_gate_block_top', 'next_gate_block_bottom'])
All actions [119, None]
State: [125.    0.   34.5  90.5 202.  130.  180. ]
Action: 119 jump!
Reward: 0.0
Action: None chill
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: None chill
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: None chill
Reward: 0.0
Action: None chill
Reward: 1.0
Action: None chill
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: None chill
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: None chill
Reward: 0.0
Action: None chill
Reward: 1.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Action: 119 jump!
Reward: 0.0
Actio

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 27ms/step
Prediction [0.10938094 0.04945939]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 30ms/step
Prediction [0.11946036 0.05530423]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 28ms/step
Prediction [0.16066688 0.07205488]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 30ms/step
Prediction [0.22083819 0.09493819]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 25ms/step
Prediction [ 0.0430377  -0.00224664]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 7ms/step - loss: 0.5242 - accuracy: 0.8281
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 26ms/step
Prediction [ 0.02810724 -0.01552059]
Action: None chill
Reward: 0.0
1/1 [==============================] - 0s 31ms/step
Prediction [ 0.01096085 -0.03186867]
Action: None chill
Reward: 0.0
1

KeyboardInterrupt: 